## Deliverable 2. Create a Customer Travel Destinations Map.

In [137]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from configuration import map_api_key
#
# Configure gmaps API key
gmaps.configure(api_key=map_api_key)


In [28]:
# 1. Import the WeatherPy_database.csv file.

city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.15,-70.92,55.40,71,40,5.82,scattered clouds
1,1,Butaritari,KI,3.07,172.79,82.08,77,34,7.58,light rain
2,2,Cape Town,ZA,-33.93,18.42,57.99,76,0,10.29,clear sky
3,3,Goderich,CA,43.75,-81.72,69.80,60,100,18.34,overcast clouds
4,4,Hilo,US,19.73,-155.09,82.40,58,90,6.93,overcast clouds


In [39]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Please enter your minimum temperature: '))
max_temp = float(input('Please enter your maximum temperature: '))


In [63]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] < max_temp)]
preferred_cities.head(10)

(121, 10)

In [58]:
# 4a. Determine if there are any empty rows.
null_counts = preferred_cities.isnull().sum()
null_counts

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [59]:
# Print out the columns that have null values
null_counts[null_counts > 0].sort_values(ascending=False)



Series([], dtype: int64)

In [64]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.07,172.79,82.08,77,34,7.58,light rain
4,4,Hilo,US,19.73,-155.09,82.40,58,90,6.93,overcast clouds
15,15,Port Macquarie,AU,-31.43,152.92,82.99,41,0,1.99,clear sky
20,20,Kapaa,US,22.08,-159.32,82.40,69,75,17.22,broken clouds
25,25,Atuona,PF,-9.80,-139.03,78.01,76,9,17.09,clear sky
...,...,...,...,...,...,...,...,...,...,...
685,685,Merauke,ID,-8.47,140.33,86.97,59,100,10.31,overcast clouds
692,692,Tual,ID,-5.67,132.75,83.91,77,100,12.86,moderate rain
698,698,Ligayan,PH,4.65,119.47,83.17,68,51,0.96,broken clouds
699,699,Marawi,PH,8.00,124.29,80.47,64,100,3.80,overcast clouds


In [118]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,82.08,light rain,3.07,172.79,
4,Hilo,US,82.40,overcast clouds,19.73,-155.09,
15,Port Macquarie,AU,82.99,clear sky,-31.43,152.92,
20,Kapaa,US,82.40,broken clouds,22.08,-159.32,
25,Atuona,PF,78.01,clear sky,-9.80,-139.03,
26,Kirakira,SB,86.16,light rain,-10.45,161.92,
33,Meulaboh,ID,83.21,broken clouds,4.14,96.13,
35,Camacari,BR,80.60,broken clouds,-12.70,-38.32,
40,Bengkulu,ID,82.47,broken clouds,-3.80,102.27,
44,Sao Filipe,CV,78.66,clear sky,14.90,-24.50,


In [95]:
# params = {
#     "radius": 5000,
#     "type": "lodging",
#     "key": map_api_key,
#     "location":'14.90,-24.50'
# }
# hotel = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?',params=params).json()

In [87]:
# hotel['results'][0]['name']

'Tortuga B&B'

In [119]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": map_api_key,
    'location': ''
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    # print(params)
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url,params=params).json()
    # print(hotels['results'][0]['name'])
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except:
        print(f'Hotel not found....... Skipping {row["City"]}')
        

Hotel not found....... Skipping Manukan
Hotel not found....... Skipping Merauke
Hotel not found....... Skipping Ligayan


In [132]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ' ']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,82.08,light rain,3.07,172.79,Isles Sunset Lodge
4,Hilo,US,82.40,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
15,Port Macquarie,AU,82.99,clear sky,-31.43,152.92,Rydges Port Macquarie
20,Kapaa,US,82.40,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
25,Atuona,PF,78.01,clear sky,-9.80,-139.03,Villa Enata
...,...,...,...,...,...,...,...
678,Kulhudhuffushi,MV,82.98,overcast clouds,6.62,73.07,Haajy Guest House 1
679,Acarau,BR,80.94,scattered clouds,-2.89,-40.12,castelo pizzaria
692,Tual,ID,83.91,moderate rain,-5.67,132.75,Grand Vilia Hotel
699,Marawi,PH,80.47,overcast clouds,8.00,124.29,Derogongan Residence


In [133]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv('WeatherPy_vacation.csv', index_label="City_ID")

In [149]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [150]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))